
**Assignment 2: Agents and Environments (Given: 31 Jan 2023, Due: 14 Feb 2023 2pm)**
    

**General instructions**

* Solutions are to be typed in the `.ipynb` file provided and uploaded in the lab course page in Moodle before the due date. 
* Your code should be well commented and should be compatible with python3.
* For this assignment, you are allowed to import the libraries `pandas`, `numpy`, `random` and `re` of python3. No other libraries may be imported.

# Text Generation (Static Prediction Task)

In [ ]:
import pandas as pd
import numpy as np
import re
import random as rnd

(a) Read the text file “speeches.txt” and find the list of unique words with their frequency of occurrence.

In [ ]:
# file = open('./speeches.txt','r')
# file = open('./text.txt','r')
file = open('./text2.txt','r')

# 1a
text = file.read()
# reading the file
text = text.lower()
# converting all words to lowercase
words = text.split()
# converting text to a list containing words
words = [word.strip('$.,/!@#$%^&*()-=_+[]{}1234567890 ') for word in words]
# removing all special characters given in the argument
words = [word.replace("'s",'') for word in words]
# deleting all apostrophes
words = [word.strip('\'\"') for word in words if word!='' and word!='re']
# removing all single quotes and double quotes
words = [word for word in words if word.isalpha()]

uniqueWordList = []
# initializing uniqueWordList as empty list
for word in words:
  if word not in uniqueWordList:
    uniqueWordList.append(word)
    # we append only if the word is not in uniqueWordList

freqList = [
  words.count(word) for word in uniqueWordList
]
# table = {
#   'words':uniqueWordList,
#   'freq': freqList
# }
# print(pd.DataFrame(table))
# frequency list of each word

# print(words)
# print(uniqueWordList)
# print(freqList)



(b) Let the number of unique words be $n$. Construct an $n \times n$ matrix $M$ where the $i$th row corresponds to the $i$th word, and the $(i, j)$th entry stands for the frequency of occurrence of $j$th word after the $i$th word. Write a function, which accepts a given word, and returns the frequencies of occurrences of the next words (i.e., the row corresponding to the given word).

In [ ]:

# 1b

numUniqueWords = len(uniqueWordList)

nnmatrix = []
# initializing matrix to empty list

for i in range(numUniqueWords):
  rowToBeAppended = []
  # initialising an empty row
  for j in range(numUniqueWords):
    ith_word = uniqueWordList[i]
    jth_word = uniqueWordList[j]
    count = 0
    # starting count from 0
    for x in range(len(words)):
      if x<len(words)-1 and words[x] == ith_word and words[x+1] == jth_word:
        count += 1
        # finding frequency of occurrence of jth_word after ith_word
    rowToBeAppended.append(count)
  nnmatrix.append(rowToBeAppended)

# print(nnmatrix)
# print('[')
# for row in nnmatrix:
#   print(row)
# print(']')

def findFreqList():
  word = str(input())
  if word not in uniqueWordList:
    print('word not found')
  else:
    index = uniqueWordList.index(word)
    return nnmatrix[index]


row = findFreqList()
print(row)


(c) Use $M$ to produce a random text $T$ of length $5000$ where the first word in $T$ is a random word from the input file and the subsequent words are sampled according to $M$.

In [ ]:

# 1c

textLength = 0
word = rnd.choices(uniqueWordList,k=1)[0]
print(word,end=' ')
textLength+=1


# it has to be 5000
while textLength < 20:
  index = uniqueWordList.index(word)
  freq_dist_row = nnmatrix[index]
  word = rnd.choices(uniqueWordList,weights=freq_dist_row,k=1)[0]
  print(word,end=' ')
  textLength+=1
print()


file.close()

# Room Cleaning Robot (Dynamic Control Task)

Consider a Room Cleaner Robot which cleans a room containing dirt. A room can be treated as a $m \times n$ grid with walls on all sides. 

(a) Implement an environment which takes as inputs the values $m$ and $n$. Initially, 10 random cells in the grid contain dirt. The list $d_t$ with dimensions $m \times n$ contains the information on the amount of dirt in each cell of the grid. At any point of time, a unit dirt is added at a random location, i.e., a location $(x, y)$ is picked uniformly at random and $d_t(x, y)$ is updated to $d_t(x, y) + 1$.

In [ ]:
# 2a
class RoomFloor:
  def __init__(self,m,n) -> None:
    self.floorClock = 1
    self.isClean = False
    self.rows = m
    self.columns = n
    self.floorGrid = [
      [0 for _ in range(n)] for _ in range(m)
    ]
    # initialize 10 random locations to 1
    numDirtPlaces = 0
    while numDirtPlaces < 10:
      i = rnd.randint(0,m-1)
      j = rnd.randint(0,n-1)
      if self.floorGrid[i][j] == 0:
        self.floorGrid[i][j] = 1
        numDirtPlaces += 1

    
  def findIfClean(self) -> bool:
    for row in self.floorGrid:
      for element in row:
        if element != 0:
          return False
    return True

  def updateClock(self):
    self.floorClock += 1

  def addDirt(self):
    y = rnd.randint(0,self.rows-1)
    x = rnd.randint(0,self.columns - 1)
    # self.clock += 1
    self.updateClock()
    self.floorGrid[y][x] += 1

  def print_state_of_floor(self):
    print()
    print(f'clock of floor is {self.floorClock}')
    for row in self.floorGrid:
      for tile in row:
        print(f'  {tile}  ',end=' ')
      print()
    
    print()

  def hasDirt_at_xy(self,x,y) -> bool:
    if self.floorGrid[y][x] != 0:
      return True
    return False



m=int(input())
n=int(input())

roomFloor = RoomFloor(m,n)



(b) Implement an agent which at each time instant $t$ observes its position $o_t =(x_t, y_t)$  (but not the dirt information in this location) and performs a random action $a_t \in \{up,down,right,left, pickDirt\}$. On hitting a wall or picking dirt from the current location $(x_t, y_t)$, the agent stays in the same position $(x_t, y_t)$, otherwise, its position changes according to the action. Every action is associated with a reward (or penalty) defined as follows.

   
$$r_t = R((x_t, y_t), a_t) = 
     \begin{cases}
       -1, &\text{if the agent tries to pick dirt and } (x_t, y_t) \text{ is a clean cell}\\
      -10, &\text{if the agent hits a wall}
       \\ 
       d, &\text{if } (x_t, y_t) \text{ has } d \text{ units of dirt and the agent picks it}\\
       0, &\text{otherwise }
     \end{cases}
$$



In [ ]:


class Agent():

  def __init__(self,roomFloor: RoomFloor) -> None:
    self.agentClock = 1
    self.inRoomFloor = roomFloor
    self.loc = [0,0]
    # [x,y]
    self.actionList = ['up','down','left','right','pickDirt']
    self.currentAction = rnd.choices(
      self.actionList,
      k=1
    )[0]
    self.reward = self.calc_reward(self.currentAction,self.loc,self.inRoomFloor)
    

  def calc_reward(self,action:str,loc:list,roomFloor: RoomFloor) -> int:

    x = loc[0]
    y = loc[1]
    if action == 'pickDirt':
      if roomFloor.hasDirt_at_xy(x,y):
        amountOfDirt = roomFloor.floorGrid[y][x]
        roomFloor.floorGrid[y][x] = 0
        return amountOfDirt
      else:
        return -1
    elif action == 'up':
      if y==0:
        return -10
      else:
        # self.loc[1] -= 1

        # to move up the list , we have to decrease 1
        return 0
    elif action == 'down':
      if y==roomFloor.rows - 1:
        return -10
      else:
        # self.loc[1] += 1
        return 0
    elif action == 'left':
      if x==0:
        return -10
      else:
        # self.loc[0] -= 1
        return 0
    else:
      if x == roomFloor.columns - 1:
        return -10
      else:
        # self.loc[0] += 1
        return 0
    
  def change_position(self,action:str,loc:list,roomFloor: RoomFloor) -> None:
    x = loc[0]
    y = loc[1]
    if action == 'pickDirt':
      if roomFloor.hasDirt_at_xy(x,y):
        amountOfDirt = roomFloor.floorGrid[y][x]
        roomFloor.floorGrid[y][x] = 0
        # return amountOfDirt
      # else:
      #   return -1
    elif action == 'up':
      if y==0:
        pass
        # return -10
      else:
        self.loc[1] -= 1

        # to move up the list , we have to decrease 1
        # return 0
    elif action == 'down':
      if y==roomFloor.rows - 1:
        # return -10
        pass
      else:
        self.loc[1] += 1
        # return 0
    elif action == 'left':
      if x==0:
        # return -1
        pass
      else:
        self.loc[0] -= 1
        # return 0
    else:
      if x == roomFloor.columns - 1:
        # return -10
        pass
      else:
        # return 0
        self.loc[0] += 1



  def print_state_of_agent(self):
    print()
    print(f'The current location of agent is {self.loc}')
    print(f'The current action of agent is {self.currentAction}')
    print(f'The reward obtained is {self.reward}')
    print()

  def doAction_modifyReward(self):
    self.change_position(action=self.currentAction,loc=self.loc,roomFloor=self.inRoomFloor)
    self.currentAction = rnd.choices(
      self.actionList,
      k=1
    )[0]
    self.reward = self.calc_reward(self.currentAction,self.loc,self.inRoomFloor)
    floor = self.inRoomFloor
    floor.addDirt()
    self.agentClock = floor.floorClock

agent = Agent(roomFloor)


(c) Print out the activity at each time instant $t$ for $t = 1, \ldots, 1000$. That is, for each $t$, display the grid (with dirt values in each cell), the location of the agent, the action of the agent and the reward obtained. 

In [ ]:
count = 0
while count < 10:
  roomFloor.print_state_of_floor()
  agent.print_state_of_agent()
  agent.doAction_modifyReward()
  count += 1

# Cricket Match Simulation (Stochastic Control Task)

Consider the following simplified variant of the first innings of a one-day cricket match. The innings consists of 300 balls and at any point of time, treat the pair of batsmen playing as a single player. That is, there are 10 players: $(1, 2),(2, 3), \ldots ,(10, 11)$ with exactly one of them batting. In each ball, there are 5 possible shots $A=\{1, 2, 3, 4, 6\}$ and  each of these shots is associated with a risk of the player getting out. This risk varies from player to player. The probabilities of getting out for Player $(1, 2)$ is $pOutMin = [0.01, 0.02, 0.03, 0.1, 0.3]$ and for Player $(10, 11)$ is $pOutMax = [0.1, 0.2, 0.3, 0.5, 0.7]$,  where the $i$th entry is for the $i$th action. If there are $w$ wickets in hand, then use the formula $pOut(a, w) = pOutMax(a) - (pOutMax(a)-pOutMin(a)) \times (w- 1)/9)$ to calculate the probabilities of getting out for Player $(11-w, 11-w+1)$ where $w \in [10]$. Note that $pOut(a, 1)=pOutMin(a)$ and $pOut(a, 10)=pOutMax(a)$. Taking $pRunMin = 0.5$, and $pRunMax = 0.8$,  when the player is not getting out, the probability of successfully obtaining the runs for that shot is given by $pRun(w) = pRunMin + (pRunMax - pRunMin) \times ((w - 1)/9).$



(a) Implement an environment that maintains the state $s_t = (b_t, w_t)$ where $b_t$ is the balls left and $w_t$ is the wickets left at time $t$. Initialize the start state to $s_1 = (300, 10)$. Write a function that accepts input as $a_t \in A$ and returns $r_t$ (the runs scored on that shot) and $s_t$, and also updates $s_{t+1}$. 

In [ ]:

class Player:
  def __init__(self,index) -> None:
    # index is starting from 1
    self.indexOfPlayer = index
    self.player_tuple = (index,index+1)
    self.runs = 0
    self.isOut = False
    self.balls_played_by_thisPlayer = 0
    self.shots = [1,2,3,4,6]


    pOutMax = [0.1,0.2,0.3,0.5,0.7]
    pOutMin = [0.01,0.02,0.03,0.1,0.3]
    pOut = []
    if index==1:
      pOut = pOutMin
    elif index==10:
      pOut = pOutMax
    else:
      w = 11-index
      pOut = [
        pOutMax[i] - (pOutMax[i] - pOutMin[i])*(w-1)/9

        for i in range(5)
      ]
    
    self.pOutList = pOut

    pRunMin = 0.5
    pRunMax = 0.8
    if index==1:
      pRun = pRunMax
    elif index == 10:
      pRun = pRunMin
    else:
      w = 11 - index
      pRun = pRunMin + (pRunMax - pRunMin)*(w-1)/9
    self.pRun = pRun

  # for debugging purpose
  # printing the info of this player
  def print_state_of_player(self):
    print(f'the player index is {self.indexOfPlayer}')
    print(f'player is {self.player_tuple}')
    print(f'Runs scored is {self.runs}')
    print(f'Is the player out ? : {self.isOut}')
    print(f'probability of getting out: {self.pOutList}')
    print(f'probability of scoring runs: {self.pRun}')
    print(f'The number of balls played by this player is {self.balls_played_by_thisPlayer}')

  def player_playBall(self,a_t=0):
    a_t = int(input()) if a_t == 0 else a_t
    if a_t == 5:
      print('no such run')
      return
    self.balls_played_by_thisPlayer+=1
    # increment the number of balls played
    indexOfShot = self.shots.index(a_t)
    pOut_shot = self.pOutList[indexOfShot]
    # prob that the attempt to shoot a_t results in a wicket
    isOut = rnd.choices(
      [True,False],
      weights=[pOut_shot,1-pOut_shot],
      k=1
    )[0]
    if isOut:
      self.isOut = True
    else:
      pRun = self.pRun
      doesScore = rnd.choices(
        [True,False],
        weights=[pRun,1-pRun],
        k=1
      )[0]
      if doesScore:
        self.runs += a_t






class ODI:
  
  def __init__(self,player:Player = None) -> None:
    self.ballsPlayed = 0
    self.ballsLeft = 300
    self.totalRuns = 0
    self.players = [Player(i+1) for i in range(10)]
    # list of player
    self.isActive = True
    self.currentPlayerIndex = 1
    # first player will be player 1
    self.currentPlayerObject = self.players[self.currentPlayerIndex - 1]
    # player object of the current player

    self.wicketsTaken = 0
    self.wicketsInHand = 10
    self.currentState = (self.ballsLeft,self.wicketsInHand)


  def print_state_of_odi(self):
    print(f'The number of balls played and balls remaining is {self.ballsPlayed} and {self.ballsLeft} respectively')
    print(f'Total Runs: {self.totalRuns}')
    print(f'The current player index is {self.currentPlayerIndex}')
    print(f'the current player is {self.currentPlayerObject.player_tuple}')
    print(f'wickets taken: {self.wicketsTaken}')
    print(f'wickets in hand: {self.wicketsInHand}')
    print(f'the current state of odi: {self.currentState}')

  def ballPlayed(self,whoPlayedTheBall: Player,runs_scored = 0,is_wicket = False):
    # whoPlayedTheBall is a player object
    if self.ballsPlayed == 300:
      self.isActive = False
      return
    
    self.ballsPlayed += 1
    self.ballsLeft -= 1
    whoPlayedTheBall.balls_played_by_thisPlayer+=1

    if is_wicket:
      # if wicket happens
      self.wicketsTaken += 1
      self.wicketsInHand -= 1
      # what if current player index crosses 10
      if self.currentPlayerIndex == 10:
        self.isActive = False
        return
      self.currentPlayerIndex += 1
      self.currentPlayerObject = self.players[self.currentPlayerIndex - 1]
      whoPlayedTheBall.isOut = True
      self.currentState = (self.ballsLeft,self.wicketsInHand)


    else:
      whoPlayedTheBall.runs += runs_scored
      self.totalRuns += runs_scored


# 3a
def playBall(odi: ODI,a_t = 0) -> tuple:
  a_t = int(input()) if a_t == 0 else a_t
  if a_t == 5:
    print('no such run')
    return (0,odi.currentState)
  playerObject = odi.currentPlayerObject
  indexOfShot = playerObject.shots.index(a_t)
  # print(indexOfShot)
  
  pOutList  = playerObject.pOutList
  # pRun = playerObject.pRun
  # print(pOutList)
  isOut = rnd.choices(
    [True,False], 
    weights=[pOutList[indexOfShot],1-pOutList[indexOfShot]],
    k=1
  )[0]
  if isOut:
    odi.ballPlayed(playerObject,is_wicket=True)
    return (0,odi.currentState)
  else:
    pRun = playerObject.pRun
    doesScoreRun = rnd.choices(
      [True,False],
      weights=[pRun,1-pRun],
      k=1
    )[0]
    if not doesScoreRun:
      # dot ball
      odi.ballPlayed(playerObject)
      return (0,odi.currentState)
    else:
      runsScored = a_t
      odi.ballPlayed(playerObject,runs_scored=runsScored)
      return (runsScored,odi.currentState)


(b) Play $a_t = 1$ for each $t$ and find the average balls played and average runs scored by Player $(1,2)$  and Player $(10,11)$.

In [ ]:

# 3b
print('question 3b')

numMatches_played_by_player = 0
runs_scored = 0
numBalls_played = 0
# take 100 matches played by the player
while numMatches_played_by_player < 100:
  player1 = Player(1)
  numBalls = 0
  while numBalls < 300 and player1.isOut == False:
    player1.player_playBall(1)
    numBalls += 1
  runs_scored += player1.runs
  numBalls_played += player1.balls_played_by_thisPlayer
  numMatches_played_by_player += 1
print(f'the average balls played by the player {player1.player_tuple} is {numBalls_played/numMatches_played_by_player}')
print(
  f'the average runs scored by player {player1.player_tuple} is {runs_scored/numMatches_played_by_player}'
)


numMatches_played_by_player = 0
runs_scored = 0
numBalls_played = 0
# take 100 matches played by the player
while numMatches_played_by_player < 100:
  player10 = Player(10)
  numBalls = 0
  while numBalls < 300 and player10.isOut == False:
    player10.player_playBall(1)
    numBalls += 1
  runs_scored += player10.runs
  numBalls_played += player10.balls_played_by_thisPlayer
  numMatches_played_by_player += 1
print(f'the average balls played by the player {player10.player_tuple} is {numBalls_played/numMatches_played_by_player}')
print(
  f'the average runs scored by player {player10.player_tuple} is {runs_scored/numMatches_played_by_player}'
)


(c) Play $a_t=6$ for each $t$ and find the average balls  played and average runs scored by Player $(1,2)$ and Player $(10,11)$.

In [ ]:

# 3c
print('question 3c')

numMatches_played_by_player = 0
runs_scored = 0
numBalls_played = 0
# take 100 matches played by the player
while numMatches_played_by_player < 100:
  player1 = Player(1)
  numBalls = 0
  while numBalls < 300 and player1.isOut == False:
    player1.player_playBall(6)
    numBalls += 1
  runs_scored += player1.runs
  numBalls_played += player1.balls_played_by_thisPlayer
  numMatches_played_by_player += 1
print(f'the average balls played by the player {player1.player_tuple} is {numBalls_played/numMatches_played_by_player}')
print(
  f'the average runs scored by player {player1.player_tuple} is {runs_scored/numMatches_played_by_player}'
)

numMatches_played_by_player = 0
runs_scored = 0
numBalls_played = 0
# take 100 matches played by the player
while numMatches_played_by_player < 100:
  player10 = Player(10)
  numBalls = 0
  while numBalls < 300 and player10.isOut == False:
    player10.player_playBall(6)
    numBalls += 1
  runs_scored += player10.runs
  numBalls_played += player10.balls_played_by_thisPlayer
  numMatches_played_by_player += 1
print(f'the average balls played by the player {player10.player_tuple} is {numBalls_played/numMatches_played_by_player}')
print(
  f'the average runs scored by player {player10.player_tuple} is {runs_scored/numMatches_played_by_player}'
)

(d) Simulate 10 matches, for different constant strategies like $a_t = 1$ for each $t$, $a_t = 2$ for each $t$, etc. and and find the average runs obtained in each of the strategies.

In [ ]:

# 3d

print('question 3d')
actionList  = [1,2,3,4,6]
for action in actionList:
  numMatches = 0
  runs = 0
  while numMatches < 10:
    odi_new = ODI()
    numBalls = 0
    while numBalls < 300 and odi_new.isActive:
      playBall(odi_new,action)
    runs += odi_new.totalRuns
    numMatches += 1

  print(f'average runs obtained in the strategy where at = {action} is {runs/10}')

